"C:\\Users\\Farhan\\Desktop\\Data Quest\\DataQuest2023\\Glucose-Data"

In [1]:
import os
import pandas as pd

# Path to the folder containing the text files
folder_path = "dataquest-nascon23/Glucose-Data/Glucose-Data"

# Create an empty list to store the dataframes
dfs = []

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Get the patient name from the filename
        patient_name = os.path.splitext(filename)[0]
        
        # Read the text file into a dataframe
        df = pd.read_csv(os.path.join(folder_path, filename), sep='\t', header=None)
        
        # Rename the columns and add the patient name and ID columns
        df.columns = ['Date', 'Time', 'Code', 'GlucoseLevel']
        df.insert(0, 'PatientID', patient_name)
        #print(df['Date'])
        # Add the dataframe to the list
        dfs.append(df)
        
# Concatenate all the dataframes into one
all_dfs = pd.concat(dfs, ignore_index=True)
all_dfs
#07-27-1991

all_dfs['Date'].fillna(pd.to_datetime('07-27-1991'))
all_dfs = all_dfs.dropna(subset=['Date'])
all_dfs.isna().sum()
# all_dfs.fillna("sample date", inplace = True)
# all_dfs.isna().sum()

PatientID        0
Date             0
Time             0
Code             0
GlucoseLevel    33
dtype: int64

In [2]:
# Count the number of NaT values in the 'Date' column
num_nans = df.isna().sum()
# print(f"Number of NaT values: {num_nans}")
all_dfs

,PatientID,Date,Time,Code,GlucoseLevel
0,60,07-13-1990,11:36,57,84
1,60,07-13-1990,11:39,33,3
2,60,07-13-1990,16:43,65,0
3,60,07-13-1990,16:44,66,0
4,60,07-13-1990,16:44,62,180
...,...,...,...,...,...
25738,20,08-29-1990,20:41,64,91
25739,20,08-30-1990,05:46,58,82
25740,20,08-30-1990,11:46,60,52
25741,20,08-30-1990,17:53,62,95


In [3]:
# cpyDT = all_dfs['Date']
# cpyDT
# for i in range(len(cpyDT)):
#     x = cpyDT[i]
#     print
    

In [4]:
from datetime import datetime
# 
#     date_string = df['Date'].iloc[i]
    
# all_dfs['Date'] = datetime.strptime(all_dfs['Date'], '%m-%d-%Y')
all_dfs['new'] = 0
for i in range(22000):#len(all_dfs)):
    x = all_dfs['Date'].iloc[i]
    #print(x)
    date_object = datetime.strptime(x, "%m-%d-%Y")
    #print(date_object)
    date_object = date_object.date()
    #print(date_object)
    newformat = date_object.strftime('%m-%d-%Y')
    #print(newformat)
    all_dfs['new'].iloc[i] = newformat
#     print(newformat)

/tmp/ipykernel_44559/1839405720.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs['new'].iloc[i] = newformat


KeyboardInterrupt: 

In [ ]:
df = all_dfs.head(22000)

In [ ]:
df['Date'] = df['new']

In [ ]:
del df['new']

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.strftime('%A')


In [ ]:
# define a function to extract the year from a datetime object
df['year'] = 0
def extract_year(dt):
    return dt.year

# for i in range(len(df['Date'])):
#     df['year'].iloc[i] = df['Date'].apply(extract_year)
df['year'] = df['Date'].apply(extract_year)

In [ ]:
df

In [ ]:
# Convert the 'Time' column to a datetime data type
df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

# Define a function to get the time of day
def get_time_of_day(row):
    hour = row['Time'].hour
    if hour >= 3 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 19:
        return 'Evening'
    else:
        return 'Night' if hour >= 19 or hour < 3 else None

# Create a new column 'time_of_day'
df['time_of_day'] = df.apply(get_time_of_day, axis=1)
df['Time'] = pd.to_datetime(df['Time'])
df['Time'] = df['Time'].dt.time

In [ ]:
df

In [ ]:
#QUARTER OF DAY
def get_quarter(row):
    if row['time_of_day'] == 'Morning':
        return 1
    elif row['time_of_day'] == 'Afternoon':
        return 2
    elif row['time_of_day'] == 'Evening':
        return 3
    elif row['time_of_day'] == 'Night':
        return 4
    else:
        return None

df['quarter'] = df.apply(get_quarter, axis=1)


In [ ]:
my_dict = {
    33: "Regular insulin dose",
    34: "NPH insulin dose",
    35: "UltraLente insulin dose",
    48: "Unspecified blood glucose measurement",
    57: "Unspecified blood glucose measurement",
    58: "Pre-breakfast blood glucose measurement",
    59: "Post-breakfast blood glucose measurement",
    60: "Pre-lunch blood glucose measurement",
    61: "Post-lunch blood glucose measurement",
    62: "Pre-supper blood glucose measurement",
    63: "Post-supper blood glucose measurement",
    64: "Pre-snack blood glucose measurement",
    65: "Hypoglycemic symptoms",
    66: "Typical meal ingestion",
    67: "More-than-usual meal ingestion",
    68: "Less-than-usual meal ingestion",
    69: "Typical exercise activity",
    70: "More-than-usual exercise activity",
    71: "Less-than-usual exercise activity",
    72: "Unspecified special event"
}


In [ ]:
# Loop through the keys in the dictionary and assign 1 if the key is present in the code column, else 0
for key in my_dict.keys():
    df[my_dict[key]] = df['Code'].apply(lambda x: 1 if x == key else 0)


In [ ]:
df['time_since_beginning'] = df['time_since_beginning'].abs()

# set time_since_beginning to 0 whenever PatientID changes
df['time_since_beginning'] = np.where(df['PatientID'].ne(df['PatientID'].shift()), 0, df['time_since_beginning'])
df['time_since_beginning'] = df.groupby('PatientID')['time_since_beginning'].apply(lambda x: x.cumsum())
df

In [215]:
df

,PatientID,Date,Time,Code,GlucoseLevel,year,Day,time_of_day,quarter,Regular insulin dose,...,Hypoglycemic symptoms,Typical meal ingestion,More-than-usual meal ingestion,Less-than-usual meal ingestion,Typical exercise activity,More-than-usual exercise activity,Less-than-usual exercise activity,Unspecified special event,glucose_level_since_beginning,time_since_beginning
0,1,1991-04-15,1900-01-01 13:08:00,33,5,1991,Monday,Afternoon,2.000000,1,...,0,0,0,0,0,0,0,0,0,0.000000
1,1,1991-04-15,1900-01-01 17:17:00,62,41,1991,Monday,Evening,3.000000,0,...,0,0,0,0,0,0,0,0,41,14940.000000
2,1,1991-04-15,1900-01-01 18:23:00,33,6,1991,Monday,Evening,3.000000,1,...,0,0,0,0,0,0,0,0,6,63720.000000
3,1,1991-04-15,1900-01-01 18:24:00,35,3,1991,Monday,Evening,3.000000,0,...,0,0,0,0,0,0,0,0,3,165300.000000
4,1,1991-04-15,1900-01-01 21:16:00,64,119,1991,Monday,Night,4.000000,0,...,0,0,0,0,0,0,0,0,3,348960.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22025,59,1991-06-27,1900-01-01 12:00:00,60,83,1991,Thursday,Afternoon,2.000000,0,...,0,0,0,0,0,0,0,0,0,7784203113825692672.000000
22026,59,1991-06-27,1900-01-01 12:00:00,33,5,1991,Thursday,Afternoon,2.000000,1,...,0,0,0,0,0,0,0,0,0,7807310023800382464.000000
22027,59,1991-06-27,1900-01-01 17:50:00,62,193,1991,Thursday,Evening,3.000000,0,...,0,0,0,0,0,0,0,0,0,7830420111922025472.000000
22028,59,1991-06-27,1900-01-01 17:50:00,33,11,1991,Thursday,Evening,3.000000,1,...,0,0,0,0,0,0,0,0,0,7853533378482416640.000000


In [206]:
df['glucose_level_since_beginning'] = 0

# Looping through each row in the dataframe and updating the new column based on the conditions
current_patient_id = df.loc[0, 'PatientID']
for i in range(1, len(df)):
    if df.loc[i, 'PatientID'] != current_patient_id:
        df.loc[i, 'glucose_level_since_beginning'] = 0
        current_patient_id = df.loc[i, 'PatientID']
    elif df.loc[i, 'Code'] in [61, 62, 33, 34, 35, 59, 63, 67, 68]:
        df.loc[i, 'glucose_level_since_beginning'] = df.loc[i, 'GlucoseLevel']
    else:
        df.loc[i, 'glucose_level_since_beginning'] = df.loc[i-1, 'glucose_level_since_beginning']


C:\Users\Farhan\AppData\Local\Temp\ipykernel_11600\1431799604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['glucose_level_since_beginning'] = 0
C:\Users\Farhan\AppData\Local\Temp\ipykernel_11600\1431799604.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'glucose_level_since_beginning'] = df.loc[i, 'GlucoseLevel']


KeyError: 2357

In [190]:
df.head()

,PatientID,Date,Time,Code,GlucoseLevel,year,Day,time_of_day,quarter,Regular insulin dose,...,Pre-snack blood glucose measurement,Hypoglycemic symptoms,Typical meal ingestion,More-than-usual meal ingestion,Less-than-usual meal ingestion,Typical exercise activity,More-than-usual exercise activity,Less-than-usual exercise activity,Unspecified special event,glucose_level_since_beginning
0,1,1991-04-15,13:08:00,33,5,1991,Monday,Afternoon,2.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1991-04-15,17:17:00,62,41,1991,Monday,Evening,3.0,0,...,0,0,0,0,0,0,0,0,0,41
2,1,1991-04-15,18:23:00,33,6,1991,Monday,Evening,3.0,1,...,0,0,0,0,0,0,0,0,0,6
3,1,1991-04-15,18:24:00,35,3,1991,Monday,Evening,3.0,0,...,0,0,0,0,0,0,0,0,0,3
4,1,1991-04-15,21:16:00,64,119,1991,Monday,Night,4.0,0,...,1,0,0,0,0,0,0,0,0,3


In [195]:
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')


C:\Users\Farhan\AppData\Local\Temp\ipykernel_11600\1341322729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')


In [222]:
df = df.dropna()

df['GlucoseLevel'] = df['GlucoseLevel'].astype(int)


ValueError: invalid literal for int() with base 10: "0''"

In [219]:
df.to_csv('output.csv', index=False)

In [202]:
df.isna().sum()

PatientID                                    0
Date                                         0
Time                                         7
Code                                         0
GlucoseLevel                                30
year                                         0
Day                                          0
time_of_day                                  7
quarter                                      7
Regular insulin dose                         0
NPH insulin dose                             0
UltraLente insulin dose                      0
Unspecified blood glucose measurement        0
Pre-breakfast blood glucose measurement      0
Post-breakfast blood glucose measurement     0
Pre-lunch blood glucose measurement          0
Post-lunch blood glucose measurement         0
Pre-supper blood glucose measurement         0
Post-supper blood glucose measurement        0
Pre-snack blood glucose measurement          0
Hypoglycemic symptoms                        0
Typical meal 